### US Income Data

What I want:
</br>
<img width='500px' src="target_sketch.jpeg"></img>
</br>
What I have:
</br>

<img width='500px' src="start_shot.png"></img>
</br>
What I need to do:
* Load in each sheet to temp dfs
* Melt
* Add series and view cols

In [2]:
import pandas as pd
import numpy as np

wd = '/Users/finn/Documents/GitHub/data/e4e3' #Path of e4e3 folder

#### First the Income by Percentiles:

In [72]:
df = pd.read_excel(f'{wd}/Uncleaned Data/US Quintiles Incomes 1967-2021.xlsx',skiprows=68)
df.columns = ['Year', 'n', '20th', '40th', '60th', '80th', '95th']
df['Year'] = df['Year'].astype(str).str[0:4] # Some Years have notes, let's lose these
df = df
df = df.loc[:, df.columns != 'n'] #don't care about n
df = pd.melt(df, id_vars=['Year'], var_name='Series', value_name='Income')
df['View'] = 'by Percentile'

pctile_df = df
pctile_df

,Year,Series,Income,View
0,2021,20th,28007,by Percentile
1,2020,20th,28544,by Percentile
2,2019,20th,29762,by Percentile
3,2018,20th,27621,by Percentile
4,2017,20th,27435,by Percentile
...,...,...,...,...
280,1971,95th,144394,by Percentile
281,1970,95th,144708,by Percentile
282,1969,95th,142828,by Percentile
283,1968,95th,135852,by Percentile


#### Then by Race:
Have one sheet with lots of tables. Let's define a list of lists to store race and row number for each then loop over. 

In [73]:
rows_races = [
    ['All Races', 8],
    ['White Alone', 68],
    ['White', 93],
    ['White Alone, Not Hispanic', 131],
    ['White, Not Hispanic', 156],
    ['Black Alone or in Combination', 189],
    ['Black Alone', 214],
    ['Asian Alone or in Combination', 277],
    ['Asian Alone', 302],
    ['Asian and Pacific Islander', 327],
    ['Hispanic (any race)', 345]] #Used to loop over document, selecting each race-income table

In [143]:
race_df = None
for race, row in rows_races:
    df = pd.read_excel(f'{wd}/Uncleaned Data/US Median Income By Year and Race 1967-.xlsx',skiprows=row)
    df.columns = ['Year', 'n', 'Median (Current USD)', 'Income',  'Mean (Current USD)', 'Mean (2021 USD)']
    df = df[['Year', 'Income']] #Only care about current dollars real incomes
    df['Year'] = df['Year'].astype(str).str[0:4] # Some Years have notes, let's lose these
    df['Series'] = race
    #We only want to read until the end of the current table not the whole sheet, so:
    if len(df.index[df.iloc[:, 1].isna()])>0: #Check if there are any empty rows [end of table]
        first_empty_row = df.index[df.iloc[:, 1].isna()][0] #1. Find first empty row if there are
        df = df.iloc[:first_empty_row, :] #2. lose all after that
    if race_df is None:
        race_df = df
    else:
        race_df = pd.concat([race_df, df], ignore_index=True)
#Have some (~5 rows) missing data, let's lose those rows
race_df = race_df[race_df['Income'] != 'N']
race_df['View'] = 'by Race'


<b> Binding pre-2001 racial categories to post-2021 racial categories </b>

In 2001, racial categories changed. Let's reduce down to a simple (oversimple?) set: White (Non Hispanic), Hispanic, Black, Asian and Pacific Islander

In [144]:
rebinds = {
    'White Alone, Not Hispanic' : 'White, Not Hispanic',
    'Black Alone or in Combination' : 'Black',
    'Asian and Pacific Islander': 'Asian'
}

In [153]:
race_df.replace({"Series":rebinds})

,Year,Income,Series,View
0,2021,70784,All Races,by Race
1,2020,71186,All Races,by Race
2,2019,72808,All Races,by Race
3,2018,68168,All Races,by Race
4,2017,67571,All Races,by Race
...,...,...,...,...
316,1976,41545,Hispanic (any race),by Race
317,1975,40704,Hispanic (any race),by Race
318,1974,44253,Hispanic (any race),by Race
319,1973,44513,Hispanic (any race),by Race


In [154]:
race_df = race_df.replace({"Series":rebinds})
race_df = race_df[race_df['Series'].isin(list(rebinds.values())+['Hispanic (any race)'])]

#

#### and finally by Region:
Have one sheet with lots of tables. Let's define a list of lists to store race and row number for each then loop over. 

In [156]:
rows_regions = [
    ['United States', 8],
    ['Northeast', 60],
    ['Midwest', 112],
    ['South', 164],
    ['West', 216]]

In [157]:
region_df = None
for region, row in rows_regions:
    df = pd.read_excel(f'{wd}/Uncleaned Data/US Median Incomes by Region.xlsx',skiprows=row)
    df.columns = ['Year', 'n', 'Median (Current USD)', 'Income',  'Mean (Current USD)', 'Mean (2021 USD)']
    df = df[['Year', 'Income']] #Only care about current dollars real incomes
    df['Year'] = df['Year'].astype(str).str[0:4] # Some Years have notes, let's lose these
    df['Series'] = region
    #We only want to read until the end of the current table not the whole sheet, so:
    if len(df.index[df.iloc[:, 1].isna()])>0: #Check if there are any empty rows [end of table]
        first_empty_row = df.index[df.iloc[:, 1].isna()][0] #1. Find first empty row if there are
        df = df.iloc[:first_empty_row, :] #2. lose all after that
    if region_df is None:
        region_df = df
    else:
        region_df = pd.concat([region_df, df], ignore_index=True)
#Have some (~5 rows) missing data, let's lose those rows
region_df = region_df[region_df['Income'] != 'N']
region_df['View'] = 'by Region'

### Merging Them All
and exporting to a csv

In [158]:
df = pd.concat([pctile_df, race_df, region_df])
df.to_csv('us_incomes_pctile_race_region.csv', index=False)
df.to_excel('us_incomes_pctile_race_region.xlsx', index=False)